In [10]:
import re
import string

import pandas as pd

In [11]:
name = 'Book1.xlsx - Raw Text.csv'
df = pd.read_csv('docs/{}'.format(name), header=None)

In [12]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
# Use nice Pandas functionality to create a new column, without spaces
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def replace_punctuation_and_spaces(text):
    text = remove_punctuation(text)
    # Space not a part of string.punctuation, so we do it manually.
    text = text.replace(' ', '')
    return text

# Count words and characters
def count_words(text):
    text = remove_punctuation(text)
    text = remove_numbers(text)
    # Split on whitespace.
    split_text = text.split()
    return len(split_text)

def remove_numbers(text):
    for number in '1234567890':
        text = text.replace(number, '')
    return text

def format_speech(spreadsheet_row):
    text = spreadsheet_row.iloc[0]
    index = spreadsheet_row['index']
    # \n is the newline sign, split the text on that.
    splitted_text = text.split('\n')
    
    # Remove empty lines
    no_empty_lines = [t for t in splitted_text if t.strip() != '']

    # Remove whitespace in the beginning of a line
    no_initial_whitespace = [t.lstrip() for t in no_empty_lines]
    
    # Put into a Pandas dataframe
    df = pd.DataFrame({'text': no_initial_whitespace})

    # Add empty columns
    df['References to One or More External Actor'] = ''
    df['Reference to Wider Community (including "us" and key comparison other)'] = ''
    df['External Actor Code'] = ''

    
    df['no_formatting'] = df.apply(lambda row: replace_punctuation_and_spaces(row['text']), axis=1)
    # count words
    df['word_count'] = df.apply(lambda row: count_words(row['text']), axis=1)    
    
    # Count characters
    df['char_count'] = df.apply(lambda row: len(remove_numbers(row['no_formatting'])), axis=1)
    df.index = range(1, len(df) + 1)
    df.to_csv('formatted/formatted_speech_{}.csv'.format(index))

In [14]:
# Does "format_speech" once for every row.
df['index'] = df.index
df.apply(format_speech, axis=1);

In [15]:
# Sanity check
pd.read_csv('formatted/formatted_speech_0.csv')

,Unnamed: 0,text,References to One or More External Actor,"Reference to Wider Community (including ""us"" and key comparison other)",External Actor Code,no_formatting,word_count,char_count
0,1,친애하는 동지들 ! 동포형제자매들 !,NaN,NaN,NaN,친애하는동지들동포형제자매들,3,14
1,2,오늘 우리는 영웅적인 투쟁과 위훈으로 빛나는 1993년을 보내고 신심과 락관에 넘쳐...,NaN,NaN,NaN,오늘우리는영웅적인투쟁과위훈으로빛나는1993년을보내고신심과락관에넘쳐새해1994년을맞이합니다,14,41
2,3,나는 새해를 맞으면서 당과 혁명에 대한 끝없는 충성심을 지니고 주체의 사회주의위업을...,NaN,NaN,NaN,나는새해를맞으면서당과혁명에대한끝없는충성심을지니고주체의사회주의위업을위하여헌신분투하고있...,26,97
3,4,나는 조국의 평화적통일을 위하여 적극 투쟁하고있는 남녘의 형제들과 재일동포들을 비롯...,NaN,NaN,NaN,나는조국의평화적통일을위하여적극투쟁하고있는남녘의형제들과재일동포들을비롯한해외의모든우리겨...,18,67
4,5,나는 정의와 평화를 귀중히 여기며 자주적인 새 세계를 지향해나가는 5대륙의 진보적인...,NaN,NaN,NaN,나는정의와평화를귀중히여기며자주적인새세계를지향해나가는5대륙의진보적인민들과벗들에게새해의...,15,52
5,6,1993년은 우리 당의 자주적인 혁명로선의 정당성과 주체조선의 위력을 힘있게 과시한...,NaN,NaN,NaN,1993년은우리당의자주적인혁명로선의정당성과주체조선의위력을힘있게과시한력사적인해였습니다,12,42
6,7,"지난해에 우리 공화국을 고립, 질식시키고 우리의 사회주의위업을 압살하려는 제국주의자...",NaN,NaN,NaN,지난해에우리공화국을고립질식시키고우리의사회주의위업을압살하려는제국주의자들과반동들의책동은...,21,84
7,8,조국과 혁명 앞에 엄혹한 시련이 닥쳐왔을 때 우리 당은 조성된 정세에 대처하여 전국...,NaN,NaN,NaN,조국과혁명앞에엄혹한시련이닥쳐왔을때우리당은조성된정세에대처하여전국전민전군에준전시상태를선...,35,131
8,9,당의 전투적호소를 심장으로 받아안은 우리의 영용한 인민군장병들과 전체 인민들은 필승...,NaN,NaN,NaN,당의전투적호소를심장으로받아안은우리의영용한인민군장병들과전체인민들은필승의신념을가지고충천...,37,146
9,10,"이것은 우리당의 혁명로선의 승리이며 수령, 당, 대중의 일심단결의 위대한 승리입니다.",NaN,NaN,NaN,이것은우리당의혁명로선의승리이며수령당대중의일심단결의위대한승리입니다,10,35
